# Segment SDS Archive based on launch event times

Launch times come from 'PilotStudy_KSC_Rocket_Launches.xlsx'
SDS archives are at SDS_TOP and contain data from wells 6I and 6S and from seismo-acoustic stations
Segmented event waveform files are saved as MiniSEED to EVENT_DIR

## 1. Read launch data into a DataFrame and generate a list of launch times in Python datetime.datetime format

In [1]:
%run header.ipynb
EVENT_DIR = os.path.join(paths['outdir'], 'EVENTS') # must exist, and Excel file must be here
csv_launches = os.path.join(paths['outdir'], 'PilotStudy_KSC_Rocket_Launches.csv')
launchesDF = LLE.removed_unnamed_columns(pd.read_csv(csv_launches, index_col=None))
dt_tmp = pd.to_datetime(launchesDF['Date'] + ' ' +  launchesDF['Time'])
launchesDF['Date'] = [pdt.to_pydatetime() for pdt in dt_tmp]
launchesDF.drop(labels='Time', axis=1, inplace=True)
print(launchesDF)

                  Date  SLC                                    Rocket_Payload  \
0  2022-04-01 16:24:16   40                  Falcon 9 Block 5 | Transporter 4   
1  2022-04-08 15:17:12  39A  Falcon 9 Block 5 | F9-147 Axiom Mission 1 (Ax-1)   
2  2022-04-21 17:51:40   40            Falcon 9 Block 5 | Starlink Group 4-14   
3  2022-04-27 07:52:55  39A                  Falcon 9 Block 5 | SpaceX Crew-4   
4  2022-04-29 21:27:10   40            Falcon 9 Block 5 | Starlink Group 4-16   
5  2022-05-06 09:42:00  39A            Falcon 9 Block 5 | Starlink Group 4-17   
6  2022-05-14 20:40:50   40            Falcon 9 Block 5 | Starlink Group 4-15   
7  2022-05-18 10:59:40  39A            Falcon 9 Block 5 | Starlink Group 4-18   
8  2022-05-19 22:54:47   41              Atlas V N22 | AV-082 Starliner OFT-2   
9  2022-05-25 18:35:00   40                  Falcon 9 Block 5 | Transporter 5   
10 2022-06-08 21:04:00   40             Falcon 9 Block 5 | F9-157 Nilesat-301   
11 2022-06-12 17:43:00   46 

## 2. For each launch, segment raw SDS data to multi-trace MiniSEED file in Events directory

In [2]:
#import matplotlib.pyplot as plt

def event_sds2pkl(launchtime, thisSDSobj, EVENT_DIR):    
    pretrig = 3600
    posttrig = 3600
    pklfile = os.path.join(EVENT_DIR, 'launch_%s_raw.pkl' % launchtime.strftime('%Y%m%dT%H%M%S'))
    if os.path.exists(pklfile):
        print('%s already exists' % pklfile)
    else:
        print('%s not found. Segmenting from SDS.' % pklfile) 
        startt = launchtime - pretrig
        endt = launchtime + posttrig
        thisSDSobj.read(startt, endt, speed=1)
        st = thisSDSobj.stream
        #st.merge(method=0,fill_value=0)
        if len(st)>0:
            try:
                st.write(pklfile, format='pickle')
            except:
                pass
        else:
            print('Got no data')
            pklfile = None
    return pklfile

def clean(st):
    for tr in st:
        if tr.stats.network != 'FL':
            continue
        tr.detrend('linear')
        tr.filter('highpass', freq=0.2, corners=2) 
        
def apply_calibration_correction(st):
    # calibration correction

    for tr in st:
        if 'countsPerUnit' in tr.stats:
            continue
        else:
            if tr.stats.network[0] =='6': # well data
                if tr.stats.channel[2] == 'D':
                    tr.stats.countsPerUnit = 1/LLE.psi2feet(1) # counts (psi) per feet
                    tr.stats.units = 'feet'
                elif tr.stats.channel[2] == 'H':
                    tr.stats.countsPerUnit = 1 # counts (Pa) per Pa
                    tr.stats.units = 'Pa'
            elif tr.stats.channel[1]=='D':
                tr.stats.countsPerUnit = 720 # counts/Pa on 1 V FS setting
                if tr.id[:-1] == 'FL.BCHH3.10.HD':
                    if tr.stats.starttime < UTCDateTime(2022,5,26): # Chaparral M25. I had it set to 1 V FS. Should have used 40 V FS. 
                        if tr.id == 'FL.BCHH3.10.HDF':
                            tr.stats.countsPerUnit = 8e5 # counts/Pa
                        else:
                            tr.stats.countsPerUnit = 720 # counts/Pa 
                    else: # Chaparral switched to 40 V FS
                        if tr.id == 'FL.BCHH3.10.HDF':
                            tr.stats.countsPerUnit = 2e4 # counts/Pa
                        else:
                            tr.stats.countsPerUnit = 18 # counts/Pa 
                tr.stats.units = 'Pa'

            elif tr.stats.channel[1]=='H':
                tr.stats.countsPerUnit = 3e8 # counts/(m/s)
                tr.stats.units = 'm/s'
            tr.data = tr.data/tr.stats.countsPerUnit
    
def maxamp(tr):
    return np.max(np.abs(tr.data))

def remove_spikes(st):
    SEISMIC_MAX = 0.1 # m/s
    INFRASOUND_MAX = 3000 # Pa
    FEET_MAX = 21 # feet
    #SEISMIC_MIN = 1e-9
    #INFRASOUND_MIN = 0.01
    
    for tr in st:
        ma = maxamp(tr)
        if tr.stats.units == 'm/s':
            tr.data[tr.data > SEISMIC_MAX] = np.nan
            tr.data[tr.data < -1 * SEISMIC_MAX] = np.nan             
        elif tr.stats.units == 'Pa':
            tr.data[tr.data > INFRASOUND_MAX] = np.nan
            tr.data[tr.data < -1 * INFRASOUND_MAX] = np.nan   
        elif tr.stats.units == 'feet':
            tr.data[tr.data > FEET_MAX] = np.nan
            tr.data[tr.data < -1 * FEET_MAX] = np.nan               

from obspy.signal.trigger import coincidence_trigger
from pprint import pprint
import matplotlib.dates as dates
def detectEvent(st, launchtime):
    trig = coincidence_trigger("recstalta", 3.5, 1, st, 3, sta=2, lta=40)
    best_trig = {}
    best_product = 0
    for this_trig in trig:
        thistime = dates.date2num(this_trig['time'])
        this_product = this_trig['coincidence_sum']*this_trig['duration']
        if this_product > best_product:
            best_trig = this_trig
            best_product = this_product
    pprint(best_trig)
    return best_trig['time']


def add_snr(st, assoctime, threshold=1.5):
    nstime = max([st[0].stats.starttime, assoctime-240])
    netime = min([st[0].stats.endtime, assoctime-60])
    sstime = assoctime
    setime = min([st[0].stats.endtime, assoctime+120])    
    for tr in st:
        tr_noise = tr.copy().trim(starttime=nstime, endtime=netime)
        tr_signal = tr.copy().trim(starttime=sstime, endtime=setime)
        tr.stats['noise'] = np.nanmedian(np.abs(tr_noise.data))
        tr.stats['signal'] = np.nanmedian(np.abs(tr_signal.data))
        tr.stats['snr'] = tr.stats['signal']/tr.stats['noise']



thisSDSobj = SDS.SDSobj(paths['SDS_TOP'])

if not 'rawfile' in launchesDF.columns:
    launchesDF['rawfile'] = None
if not 'corrected' in launchesDF.columns:
    launchesDF['corrected'] = None   
if not 'detected' in launchesDF.columns:
    launchesDF['detected'] = None  
for i, row in launchesDF.iterrows():
    launchTime = UTCDateTime(row['Date'])
    print('Processing launch at %s' % launchTime.strftime('%Y-%m-%d %H:%M:%S'))                         
    if not row['corrected']:
        if not row['rawfile']:
            rawfile = event_sds2pkl(launchTime, thisSDSobj, EVENT_DIR)
            launchesDF.at[i, 'rawfile'] = rawfile
        try:
            st = read(rawfile)    
            #st.merge(method=0, fill_value=0)
        except:
            st = Stream()
        print('%s: %d channels' % (rawfile,len(st)))

        # all these functions safe for well traces too
        clean(st) 
        apply_calibration_correction(st)
        remove_spikes(st)
        
        # write corrected event out
        correctedfile =  os.path.join(EVENT_DIR, 'launch_%s_corrected.pkl' % launchTime.strftime('%Y%m%dT%H%M%S'))
        print('Writing %s' % correctedfile)
        try:
            st.write(correctedfile, format='PICKLE') # save 2-hour event waveforms
        except:
            pass

        # subset out the seismo-acoustic traces for detection purposes
        if not row['detected']:
            SA = Stream()
            for tr in st:
                if tr.stats.network[0] != '6':
                    SA.append(tr)
            if len(SA)==0:
                continue

            assocTime = detectEvent(SA, launchTime)
            
            if abs(assocTime-launchTime)>100:
                assocTime=launchTime
            launchesDF.at[i, 'detected'] = assocTime

        else:
            assocTime = row['detected']

        
        # save 3-minute event waveforms
        st_short = st.copy().trim(starttime=assocTime-30, endtime=assocTime+150)
        print(st)
        print(assocTime)
        print(st_short)
        st_short.write(correctedfile.replace('.pkl', '_short.pkl'), format='PICKLE')

        '''
        #detectedfile =  os.path.join(EVENT_DIR, 'launch_%s_detected.pkl' % launchTime.strftime('%Y%m%dT%H%M%S'))
        #print('Writing %s' % detectedfile)
        #st.write(detectedfile, format='PICKLE') 
        

        # plot st_short
        fig, axs = plt.subplots(3)
        fig.suptitle('Vertically stacked subplots')
        for tr in st_short:
            if tr.stats.channel[1:] == 'DF':
                tr.detrend('linear')
                axs[0].plot(tr.times(), tr.data)
            elif tr.stats.channel[1:] == 'DD':
                axs[1].plot(tr.times(), tr.data)
            elif tr.stats.channel[1:] == 'DD':
            
        axs[1].plot(x, -y)
        '''
 
        # loop over stations, plotting all traces
        stations = list(set([tr.stats.station for tr in st]))
        for station in stations:
            station_stream = st_short.copy().select(station=station)
            station_stream.plot(equal_scale=False, outfile=correctedfile.replace('.pkl', '_%s.png' % station))
del thisSDSobj
launchesDF.to_csv(csv_launches.replace('.csv','_detected.csv'))          

Processing launch at 2022-04-01 16:24:16
/home/thompsong/work/PROJECTS/KSC_EROSION/EVENTS/launch_20220401T162416_raw.pkl not found. Segmenting from SDS.
st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/FL/BCHH2/HD4.D/FL.BCHH2.10.HD4.D.2022.091")
st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/FL/BCHH2/HD5.D/FL.BCHH2.10.HD5.D.2022.091")
st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/FL/BCHH2/HD6.D/FL.BCHH2.10.HD6.D.2022.091")
st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/FL/BCHH2/HD7.D/FL.BCHH2.10.HD7.D.2022.091")
st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/FL/BCHH2/HD8.D/FL.BCHH2.10.HD8.D.2022.091")
st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/FL/BCHH2/HD9.D/FL.BCHH2.10.HD9.D.2022.091")
st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/FL/BCHH3/HHE.D/FL.BCHH3.00.HHE.D.2022.091")
st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/FL/BCHH3/HHN.D/FL.BCHH3.00.HHN.D.2022.091")
st = read("/shares/hal9000/RAID

/home/thompsong/miniconda3/envs/iceweb/lib/python3.10/site-packages/obspy/io/mseed/headers.py:823: InternalMSEEDWarning: readMSEEDBuffer(): Not a SEED record. Will skip bytes 9673728 to 9673855.
  warnings.warn(_w, InternalMSEEDWarning)
/home/thompsong/miniconda3/envs/iceweb/lib/python3.10/site-packages/obspy/io/mseed/headers.py:823: InternalMSEEDWarning: readMSEEDBuffer(): Not a SEED record. Will skip bytes 9673856 to 9673983.
  warnings.warn(_w, InternalMSEEDWarning)
/home/thompsong/miniconda3/envs/iceweb/lib/python3.10/site-packages/obspy/io/mseed/headers.py:823: InternalMSEEDWarning: readMSEEDBuffer(): Not a SEED record. Will skip bytes 9673984 to 9674111.
  warnings.warn(_w, InternalMSEEDWarning)
/home/thompsong/miniconda3/envs/iceweb/lib/python3.10/site-packages/obspy/io/mseed/headers.py:823: InternalMSEEDWarning: readMSEEDBuffer(): Not a SEED record. Will skip bytes 9674112 to 9674239.
  warnings.warn(_w, InternalMSEEDWarning)
/home/thompsong/miniconda3/envs/iceweb/lib/python3.1

st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/AM/R1E5E/EHN.D/AM.R1E5E.00.EHN.D.2022.320")
st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/AM/R1E5E/EHZ.D/AM.R1E5E.00.EHZ.D.2022.320")


/home/thompsong/miniconda3/envs/iceweb/lib/python3.10/site-packages/obspy/io/mseed/headers.py:823: InternalMSEEDWarning: readMSEEDBuffer(): Not a SEED record. Will skip bytes 9611776 to 9611903.
  warnings.warn(_w, InternalMSEEDWarning)
/home/thompsong/miniconda3/envs/iceweb/lib/python3.10/site-packages/obspy/io/mseed/headers.py:823: InternalMSEEDWarning: readMSEEDBuffer(): Not a SEED record. Will skip bytes 9611904 to 9612031.
  warnings.warn(_w, InternalMSEEDWarning)
/home/thompsong/miniconda3/envs/iceweb/lib/python3.10/site-packages/obspy/io/mseed/headers.py:823: InternalMSEEDWarning: readMSEEDBuffer(): Not a SEED record. Will skip bytes 9612032 to 9612159.
  warnings.warn(_w, InternalMSEEDWarning)
/home/thompsong/miniconda3/envs/iceweb/lib/python3.10/site-packages/obspy/io/mseed/headers.py:823: InternalMSEEDWarning: readMSEEDBuffer(): Not a SEED record. Will skip bytes 9612160 to 9612287.
  warnings.warn(_w, InternalMSEEDWarning)
/home/thompsong/miniconda3/envs/iceweb/lib/python3.1

st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/FL/BCHH2/HD4.D/FL.BCHH2.10.HD4.D.2022.320")
st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/FL/BCHH2/HD5.D/FL.BCHH2.10.HD5.D.2022.320")
st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/FL/BCHH2/HD6.D/FL.BCHH2.10.HD6.D.2022.320")
st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/FL/BCHH2/HD7.D/FL.BCHH2.10.HD7.D.2022.320")
st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/FL/BCHH2/HD8.D/FL.BCHH2.10.HD8.D.2022.320")
st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/FL/BCHH2/HD9.D/FL.BCHH2.10.HD9.D.2022.320")
st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/FL/BCHH4/HD2.D/FL.BCHH4.00.HD2.D.2022.320")
st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/FL/BCHH4/HD3.D/FL.BCHH4.00.HD3.D.2022.320")
st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/FL/BCHH4/HDF.D/FL.BCHH4.00.HDF.D.2022.320")
st = read("/shares/hal9000/RAIDZ/iceweb_products/SDS/2022/FL/BCHH4/HHE.D/FL.BCHH4.

## 3. Generate HTML index

In [4]:
HTML_DIR = os.path.join(paths['outdir'], 'HTML')
if not os.path.isdir(HTML_DIR):
    os.makedirs(HTML_DIR)
indexfile = open(os.path.join(HTML_DIR, "index.html"), "w")
print(f"Writing {indexfile}")
indexfile.write("""
<html>

<head>
<title>Launch data browser</title>
</head>

<body>

<table border=1>

<tr>
<th>Launch time</th>
<th>JulDay</th>
<th>epoch</th>
<th>Seismic</th>
<th>Infrasound</th>
<th>Well_6S</th>
<th>Well_6I</th>
</tr>

<tr>
""")


for i, row in launchesDF.iterrows():
    print('\n\n\n')
    launchTime = UTCDateTime(row['Date'])
    print('Processing launch at %s' % launchTime.strftime('%Y-%m-%d %H:%M:%S'))  
    indexfile.write("<td>%s</td>\n<td>%s</td>\n<td>%.0f</td>\n" % (launchTime, launchTime.julday, launchTime.timestamp))
    
    ## Seismic
    correctedfile =  os.path.join(EVENT_DIR, 'launch_%s_corrected_short.pkl' % launchTime.strftime('%Y%m%dT%H%M%S'))
    seismicpngfile = os.path.join(HTML_DIR, 'launch_%s_seismic.png' % launchTime.strftime('%Y%m%dT%H%M%S') )
    if not os.path.exists(seismicpngfile):
        print('Reading %s' % correctedfile)
        try:
            st = read(correctedfile, format='PICKLE')
        except:
            st = Stream()
        st_seismic = st.select(channel='HH?').select(network='FL')
        if len(st_seismic)>0:
            st_seismic.plot(equal_scale=False, outfile=seismicpngfile )            
    if os.path.exists(seismicpngfile):
        st_seismic = read(correctedfile, format='PICKLE').select(channel='HH?').select(network='FL')
        indexfile.write("<td><a href=\'%s\'>%d</a></td>\n" % (os.path.basename(seismicpngfile), len(st_seismic)))
    else:
        indexfile.write("<td>0</td>\n")
        
    ## Acoustic
    acousticpngfile = os.path.join(HTML_DIR, 'launch_%s_infrasound.png' % launchTime.strftime('%Y%m%dT%H%M%S') )
    if not os.path.exists(acousticpngfile):
        if not st: # in locals():
            print('Reading %s' % correctedfile)
            try:
                st = read(correctedfile, format='PICKLE')
            except:
                st = Stream()
        st_acoustic = st.select(channel='HD?').select(network='FL')
        
        if len(st_acoustic)>0: 
            st_acoustic.plot(equal_scale=False, outfile=acousticpngfile )
    if os.path.exists(acousticpngfile): 
        st_acoustic = read(correctedfile, format='PICKLE').select(channel='HD?').select(network='FL')
        indexfile.write("<td><a href=\'%s\'>%d</a></td>\n" % (os.path.basename(acousticpngfile), len(st_acoustic) ))
    else:
        indexfile.write("<td>0</td>\n")    
         
    # Well 6S
    for well in ['6S', '6I']:
        wellpngfile = os.path.join(HTML_DIR, 'launch_%s_well_%s.png' % (launchTime.strftime('%Y%m%dT%H%M%S'), well) )
        if not os.path.exists(wellpngfile):
            if not 'st': # in locals():
                print('Reading %s' % correctedfile)
                try:
                    st = obspy.read(correctedfile)
                except:
                    st = Stream()
            st_well = st.select(network=well)
            if len(st_well)>0: 
                st_well.plot(equal_scale=False, outfile=wellpngfile )
        if os.path.exists(wellpngfile):   
            st_well = read(correctedfile, format='PICKLE').select(network=well)
            indexfile.write("<td><a href=\'%s\'>%d</a></td>\n" % (os.path.basename(wellpngfile), len(st_well)))
        else:
            indexfile.write("<td>0</td>\n")              

        
    # comments
    #indexfile.write("<td>%s</td>\n" % launchcomments[launchnum])
    
    # end this row of table
    indexfile.write("</tr>\n\n")

indexfile.write("""

</table>

</body>

</html>
""")
indexfile.close()


Writing <_io.TextIOWrapper name='/home/thompsong/work/PROJECTS/KSC_EROSION/HTML/index.html' mode='w' encoding='UTF-8'>




Processing launch at 2022-04-01 16:24:16
Reading /home/thompsong/work/PROJECTS/KSC_EROSION/EVENTS/launch_20220401T162416_corrected_short.pkl
Reading /home/thompsong/work/PROJECTS/KSC_EROSION/EVENTS/launch_20220401T162416_corrected_short.pkl




Processing launch at 2022-04-08 15:17:12
Reading /home/thompsong/work/PROJECTS/KSC_EROSION/EVENTS/launch_20220408T151712_corrected_short.pkl




Processing launch at 2022-04-21 17:51:40
Reading /home/thompsong/work/PROJECTS/KSC_EROSION/EVENTS/launch_20220421T175140_corrected_short.pkl




Processing launch at 2022-04-27 07:52:55
Reading /home/thompsong/work/PROJECTS/KSC_EROSION/EVENTS/launch_20220427T075255_corrected_short.pkl




Processing launch at 2022-04-29 21:27:10
Reading /home/thompsong/work/PROJECTS/KSC_EROSION/EVENTS/launch_20220429T212710_corrected_short.pkl




Processing launch at 2022-05-06 09:42:00
Reading /ho

# 4. Generate event browser

In [5]:
launchpngs = sorted(glob.glob(os.path.join(HTML_DIR, "*_seismic.png")))
    
htmlstr = '''
<html>
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Launch Events</title>

    <!-- Basic style -->
    <style>
        .img-container {
            margin: 20px auto;
            width: 40%%;
        }

        .btn-container {
            text-align: center;
        }

        img {
            width: 600px;
            display: block;
            margin: auto;
            padding-bottom: 20px;
        }

        button {
            outline: none;
            padding: 12px;
            border: none;
            background-color: rgb(25, 121, 211);
            border-radius: 8px;
            color: white;
            cursor: pointer;
        }
    </style>

</head>
<body>
    <!-- HTML code for next and previous button -->
    <div class="img-container">
'''
htmlstr += f"<h1 id='imageTitle'>{os.path.basename(launchpngs[0])} </h1>"
htmlstr += f"<img src='{os.path.basename(launchpngs[0])}' class='imageTag'>"
htmlstr += '''
        <div class="btn-container">
            <button onclick="previous()">Previous</button>
            <button onclick="next()">Next</button>
        </div>
    </div>

    <!-- Javascript code for next and previous button -->
    <script>
        let images = [
'''
for launchpng in launchpngs:
    htmlstr += f"'{os.path.basename(launchpng)}',"
htmlstr += '''
        ];
        let imageTag = document.querySelector('.imageTag');
        var heading = document.getElementById('heading');
    
        let i = 0;

        function next() {
            if (i >= images.length - 1) {
                return false;
            }
            i++;
            imageTag.setAttribute('src', images[i]);
            imageTitle.textContent = images[i];
        }
        function previous() {
            if (i <= 0) {
                return false;
            }
            i--;
            imageTag.setAttribute('src', images[i]);
            imageTitle.textContent = images[i];
        }

    </script>

</body>

</html>
'''

with open(os.path.join(HTML_DIR,"launches.html"), "w") as text_file:
    text_file.write(htmlstr)

Modify above so that it looks for seismic, infrasound, and well data PNGs, and moves them all forward at once.
Perhaps we save everything to a CSV file - all the file names and any titles, descriptors, etc. Then we can load that into a dataframe.
Then within next and previous, we load appropriate row, and update things.
Could also have a clickable index in a separate frame to the left.

In [ ]:
print(launchesDF)

In [6]:
launchpngs = sorted(glob.glob(os.path.join(HTML_DIR, "*_seismic.png")))
seismicpngs = []
acousticpngs = []
wellpngs = []
for i, row in launchesDF.iterrows():
    launchTime = UTCDateTime(row['Date'])
    seismicpngs.append('launch_%s_seismic.png' % launchTime.strftime('%Y%m%dT%H%M%S') )
    acousticpngs.append('launch_%s_acoustic.png' % launchTime.strftime('%Y%m%dT%H%M%S') )
    wellpngs.append('launch_%s_well.png' % launchTime.strftime('%Y%m%dT%H%M%S') )
launchesDF['seismicpngs']=seismicpngs
launchesDF['acousticpngs']=acousticpngs
launchesDF['wellpngs']=wellpngs
ldf = launchesDF.copy()
ldf.fillna("",inplace=True)
jsonstr = ldf.to_json(orient='columns')
#from json import loads, dumps
#parsed = loads(jsonDf) # turn json string into dict
#jsonstr = dumps(parsed).replace(None, 'null') # python objects to json string

#parsed = json.loads(r)
#print(dumps(parsed, indent=4))

htmlstr = '''
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Launch Events</title>

    <!-- Basic style -->
    <style>
        .img-container {
            margin: 20px auto;
            width: 40%%;
        }

        .btn-container {
            text-align: center;
        }

        img {
            width: 600px;
            display: block;
            margin: auto;
            padding-bottom: 20px;
        }

        button {
            outline: none;
            padding: 12px;
            border: none;
            background-color: rgb(25, 121, 211);
            border-radius: 8px;
            color: white;
            cursor: pointer;
        }
    </style>

</head>
<body>
    <!-- HTML code for next and previous button -->
    <div class="img-container">
'''
htmlstr += f"        <h1 id=\"pageTitle\">{seismicpngs[0]}</h1>"
htmlstr += f"<img src=\"{seismicpngs[0]}\" class=\"imageSeismic\">"
htmlstr += '''
        <div class="btn-container">
            <button onclick="previous()">Previous</button>
            <button onclick="next()">Next</button>
        </div>
    </div>

    <!-- Javascript code for next and previous button -->
    <script>
    '''

#htmlstr += f"    const jsonObj = JSON.parse({jsonstr});"
htmlstr += f"    const jsonObj = {jsonstr};"
#htmlstr += f"    const jsonObj = {parsed};"
#htmlstr +=  "    console.log(JSON.stringify(jsonObj));"
htmlstr += f"    const seismicpngs = {seismicpngs};"
htmlstr += """

        console.log(seismicpngs);
        console.log(jsonObj['SLC']);
        let result = Object.keys(jsonObj).map(function (key) {
            return jsonObj[key];
        });
        console.log(result);
        
        let imageSeismic = document.querySelector('.imageSeismic');
        var pageTitle = document.getElementById('pageTitle');
    
        let i = 0;

        function next() {
            if (i >= seismicpngs.length - 1) {
                return false;
            }
            i++;
            imageSeismic.setAttribute('src', seismicpngs[i]);
            //pageTitle.textContent = 'seismicpngs[i]';
            document.getElementById('pageTitle').innerHTML = seismicpngs[i];
        }
        function previous() {
            if (i <= 0) {
                return false;
            }
            i--;
            imageSeismic.setAttribute('src', seismicpngs[i]);
            //pageTitle.textContent = 'seismicpngs[i]';
            document.getElementById('pageTitle').innerHTML = seismicpngs[i];
        }

    </script>

</body>

</html>
"""

with open(os.path.join(HTML_DIR,"launches2.html"), "w") as text_file:
    text_file.write(htmlstr)

/tmp/ipykernel_1930616/2419167080.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ldf.fillna("",inplace=True)


So currently I am getting my dataframe into some sort of name-value key-value hash in Javascript, but I don't have an easy way to access entire columns. And all I know how to do is take each column as an array, and then index it (as with seismicpngs). Try to find out how to index acousticpngs from Javascript key-value hash.

Also need to make RSAM plots where any zeros are turned into NaN. And then a webpage for availability plots.